In [1]:
import sys
sys.path.append('/home/yan/Data2/NOMO_Project_P1/SMPL/SMPL_python_v.1.0.0/smpl/') # smpl path
from smpl_webuser.serialization import load_model
import numpy as np
import random
import math
from scipy.optimize import minimize
from scipy.optimize import fmin
import scipy
import scipy.spatial
import pickle

smpl_model = '/home/yan/Data2/NOMO_Project_P1/SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'

# SMPL body model
model = load_model(smpl_model)

In [2]:
# Objective Function Optimization process
def meanDist(bodyParam):
    model.pose[:] = bodyParam[0:72]      # pose parameters
    model.betas[:] = bodyParam[72:82]    # shape parameters
    # vertices    :  model.r
    modelV = np.array(model.r)
    modelV = modelV - np.mean(modelV, 0)
    minY = np.min(modelV, 0)[1]
    modelV = modelV - [0, minY, 0]
    
#     # find the closest points for modelV in the targetV, calculate the mean distance
#     dist, indexes = scipy.spatial.cKDTree(targetV, leafsize=128).query(modelV)
#     dist = dist * 1000

    dist = np.sqrt(np.sum((targetV-modelV)**2, 1)) * 10000
    
    return np.mean(dist)

In [3]:
dataDir = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/'

listRange = range(0,86) + range(87, 180)
for idx in listRange:
    
    targetObj = dataDir + 'NonRigidICP_Registered_NOMO3D_Dataset1/male/male_%04d.obj' % idx

    # load target obj vertices
    with open(targetObj) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    targetV = []

    for ii in range(0, len(content)):
        x = content[ii]
        C = x.split(" ")
        if C[0] == 'v':
            CC = C[1:4]
            v = [float(vv) for vv in CC]
            targetV.append(v)
            
    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    minY = np.min(targetV, 0)[1]
    targetV = targetV - [0, minY, 0]
    
    # load the initial parameters
    initParam = dataDir + 'Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male03/male_%04d.pkl' % idx
    with open(initParam, 'r') as f:
        bodyParam0 = pickle.load(f)

    res = minimize(meanDist, bodyParam0, method='L-BFGS-B', options={'disp': True, 'ftol': 1e-20, 'gtol': 1e-20, 'maxiter': 300})

    out_res = dataDir + 'Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male05/male_%04d.pkl' % idx
    with open(out_res, 'w') as f:
        pickle.dump(res.x, f)
        
    model.pose[:] = res.x[0:72]
    model.betas[:] = res.x[72:82]
    
    registeredV = np.array(model.r)
    registeredV = registeredV - np.mean(registeredV, 0)
    minY = np.min(registeredV, 0)[1]
    registeredV = registeredV - [0, minY, 0]
    
    out_mesh = dataDir + 'Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/male05/male_%04d.obj' % idx
    with open(out_mesh, 'w') as fp:
        for v in registeredV: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
            
    if idx % 10 == 0:
        print "the " + str(idx) + "-th object prcessed, final dist = " + str(res.fun)
        

/home/yan/.local/lib/python2.7/site-packages/ipykernel_launcher.py:33: OptimizeWarning: Unknown solver options: gtol, ftol


ValueError: Jacobian is required for Newton-CG method

In [4]:
print targetV.shape

(6890, 3)
